<a href="https://www.kaggle.com/code/galenchen/taiwan-highway-vd-project?scriptVersionId=125816096" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Load the dataset and the packages
* Suggest uploading the xml.gz files onto Kaggle so it doesn't fuck up the output storage.
* Also, the package xmltodict have to be installed before anything happens.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install xmltodict
import xmltodict

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from tqdm import tqdm
import urllib.request
import gzip
from datetime import datetime, timedelta
import pytz

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## XML to JSON to DataFrame with pivoting to CSV pipeline

In [4]:
def get_vd(date, time, desired_vdid, realtime):
    
    if realtime == True:
        directory = f"VDLive_{time}.xml"
    else: directory = f"/kaggle/input/highway-vd-data/{date}/{date}/VDLive_{time}.xml/VDLive_{time}.xml"
         
    # Parse XML data into a dictionary
    with open(directory) as xml_file:
        data_dict = xmltodict.parse(xml_file.read())

    # Convert dictionary to JSON
    json_data = json.dumps(data_dict)
    #print("XML file converted to JSON.")

    # Load the JSON data into a dictionary
    data = json.loads(json_data)
    #print(data)

    # Get the desired VDID
    # Iterate through the VDLives and find the one with the desired VDID
    for vdlive in data["VDLiveList"]["VDLives"]["VDLive"]:
        if vdlive["VDID"] == desired_vdid:
            # Extract the data for the desired VDID
            link_flows = vdlive["LinkFlows"]["LinkFlow"]
            status = vdlive["Status"]
            date_time = f'{date}_{time}'

            # Print the VDID and its corresponding data
            data= {"VDID": desired_vdid, "LinkFlows": link_flows, "Status": status,
                   #"DateTime": date_time
                  }
            #print(f"Detector {desired_vdid} found.")
            #print(data)
            break

    # Extract data from JSON and flatten into a list of rows
    rows = []
    vdid = data['VDID']
    date_time = f'{date}_{time}'
    link_id = data['LinkFlows']['LinkID']

    # Situation 1: 'Lane' key is present in the 'Lanes' dictionary
    if 'Lane' in data['LinkFlows']['Lanes']:
        lanes = data['LinkFlows']['Lanes']['Lane']
        if isinstance(lanes, dict):
            # If 'Lane' is a dictionary, convert it to a list with a single element
            lanes = [lanes]
        for lane in lanes:
            lane_id = lane['LaneID']
            lane_type = lane['LaneType']
            lane_speed = lane['Speed']
            occupancy = lane['Occupancy']
            for vehicle in lane['Vehicles']['Vehicle']:
                vehicle_type = vehicle['VehicleType']
                volume = vehicle['Volume']
                speed = vehicle['Speed']
                rows.append([date_time, vdid, link_id, lane_id,
                             lane_type, lane_speed, occupancy, 
                             vehicle_type, volume, speed])

    # Situation 2: 'Lane' key is not present in the 'Lanes' dictionary
    elif 'Lane' in data['LinkFlows']['Lanes']['Lane']:
        lane = data['LinkFlows']['Lanes']['Lane']
        lane_id = lane['LaneID']
        lane_type = lane['LaneType']
        lane_speed = lane['Speed']
        occupancy = lane['Occupancy']
        for vehicle in lane['Vehicles']['Vehicle']:
            vehicle_type = vehicle['VehicleType']
            volume = vehicle['Volume']
            speed = vehicle['Speed']
            rows.append([date_time, vdid, link_id, lane_id,
                         lane_type, lane_speed, occupancy,
                         vehicle_type, volume, speed])
         
    return rows 


def get_vds(date, desired_vdid, start_time, end_time, realtime):

    # Convert start_time and end_time to integers
    start_time2 = int(start_time)
    str_end_time = end_time
    end_time = int(end_time)

    # Convert start_time and end_time to minutes
    start_time_minutes = (start_time2 // 100) * 60 + (start_time2 % 100)
    end_time_minutes = (end_time // 100) * 60 + (end_time % 100)
    total = end_time_minutes - start_time_minutes + 1

    # Create an empty list to store the results
    results = []
    missing = []
    i = 0
    current_time = start_time

    # Loop through the function multiple times and append the result to the list
    with tqdm(total=total) as pbar:
        while i < total:
            if realtime == True:
                download_xml(date, current_time)
            if file_exist==True: ##########
                result = get_vd(date, current_time, desired_vdid, realtime)
                results.extend(result)
                if realtime == True:
                    os.remove(f"/kaggle/working/VDLive_{current_time}.xml")
            else: missing.append(str(current_time))
            pbar.update(1)
            i += 1

            if int(current_time) % 100 == 59:
                current_time = str(int(current_time)+41)
            else:
                current_time = str(int(current_time)+1)
            
            if int(current_time)<10:
                current_time = f"000{current_time}"
            elif int(current_time)<100:
                current_time = f"00{current_time}"
            elif int(current_time)<1000:
                current_time = f"0{current_time}"
            else:
                current_time = current_time

    #print(results)

    # Create a pandas DataFrame from the rows list
    df = pd.DataFrame(results, columns=['DateTime', 'VDID', 'LinkID',
                                        'LaneID', 'LaneType',
                                        'LaneSpeed', 'Occupancy',
                                        'VehicleType', 'Volume', 'Speed'])
    #pivot dataframe according to vehicle
    df = df.pivot(index=['DateTime', 'VDID', 'LinkID', 'LaneID',
                     'LaneType', 'LaneSpeed', 'Occupancy'],
              columns='VehicleType',
              values=['Volume', 'Speed'])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df = df.reset_index()
    df = df.reindex(columns=['DateTime', 'VDID', #'LinkID', 
                             'LaneID',#'LaneType',
                             'LaneSpeed', 'Occupancy',
                             'Volume_S', 'Speed_S', 'Volume_L', 'Speed_L', 'Volume_T', 'Speed_T'])
    
    #pivot the dataframe according to lane
    df = df.pivot_table(index=['DateTime', 'VDID'], columns='LaneID', values=['LaneSpeed', 'Occupancy', 'Volume_S', 'Speed_S', 'Volume_L', 'Speed_L', 'Volume_T', 'Speed_T']).reset_index()
    #flatten the column names
    df.columns = ['_'.join(str(col).strip() for col in tup) for tup in df.columns.values]

    #Save to CSV
    filename = f'{date}_{desired_vdid}_{start_time}_{str_end_time}'
    df.to_csv(f'{filename}.csv', index=False)
    print(f'{filename}.csv has been saved in /kaggle/working.')
    print(f'missing files = {missing}')
    #display(df)

## Realtime downloading and conversion pipeline

Download xml.gz file, decompress into xml file, then delete the xml.gz, then feed the xml into get_vd, then delete the xml after data is analyzed.

In [5]:
file_exist = False

def download_xml(date, current_time):
    global file_exist
    
    file_exist = False
    
    url = f'https://tisvcloud.freeway.gov.tw/history/motc20/VD/{date}/VDLive_{current_time}.xml.gz'
    filename = f'VDLive_{current_time}.xml.gz'

    # Download the file
    if os.path.isfile(f"/kaggle/working/VDLive_{current_time}.xml.gz"):
        #print(f'File {filename} already exists')
        file_exist = True
    else:
        urllib.request.urlretrieve(url, filename)
        if urllib.request.urlopen(url).getcode() == 404:
            #print(f'File {filename} does not exist')
            os.remove(filename)
        elif os.path.getsize(filename) == 0:
            #print(f'File {filename} has no file size')
            os.remove(filename)
        else:
            file_exist = True
            #print(f"File {filename} downloaded")

    if file_exist:
        #file decompression        
        if os.path.isfile(f"/kaggle/working/VDLive_{current_time}.xml.gz"):
            # Step 1: Open the compressed XML file and read its contents
            with gzip.open(f'/kaggle/working/VDLive_{current_time}.xml.gz', 'rb') as f:
                xml_data = f.read()

            # Step 2: Create a new file with the same name as the compressed XML file, but with the .xml extension
            xml_file_path = f'VDLive_{current_time}.xml'

            # Step 3: Write the uncompressed XML data to the new file
            with open(xml_file_path, 'wb') as f:
                f.write(xml_data)

            # Step 4: Verify that the new file was created successfully
            if os.path.exists(xml_file_path):
                #print(f'{xml_file_path} was created successfully!')
                os.remove(f"/kaggle/working/VDLive_{current_time}.xml.gz")
            else:
                print(f'Error: {xml_file_path} was not created.')
        else:
            print(f"Error: VDLive_{current_time}.xml.gz not found.")


## The Function.
### get_vds(date, detector, start time, end time, realtime file download)

In [6]:
#get_vds("20230319", "VD-N3-N-166.688-M-RS", "1330", "1339", True )
#get_vds("20230315", "VD-N1-S-371.010-M-Loop", "1300", "1304", True )

## Acquire time so the script does automatic daily updates for yesterday.

In [7]:
# Set the time zone to GMT+8
tz = pytz.timezone('Asia/Taipei')
# Get the current time in GMT+8 time zone
now = datetime.now(tz)
# Subtract one day to get yesterday's date
yesterday = now - timedelta(days=1)
# Format yesterday's date as YYYYMMDD
yesterday_str = yesterday.strftime('%Y%m%d')

print(yesterday_str)


20230414


## Download multiple DATE x VDID loop

In [ ]:
date_list = [yesterday_str]
vdid_list = ["VD-N1-S-369.007-M-Loop", "VD-N1-S-369.400-M-Loop", "VD-N1-S-370.000-M-Loop", "VD-N1-S-371.010-M-Loop"]
start_time = "0000"
end_time = "2359"
realtime = True

for date in date_list:
    for vdid in vdid_list:
        get_vds(date, vdid, start_time, end_time, realtime)
        
print("All tasks in the loop were completed.")

 17%|█▋        | 251/1440 [10:00<1:21:23,  4.11s/it]

import os

path = "/kaggle/working/20230413_VD-N1-S-369.007-M-Loop_0000_2359.csv"
os.remove(path)